In [4]:
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

___
Team Member Names:

- Name 1: LJ Brown
- Name 2: Xingming Qu
- Name 3: Henry Warren
- Name 4: Joe McGuire

___

# In Class Assignment Two
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook)  before the end of class (or right after class). The initial portion of this notebook is given before class and the remainder is given during class. Please answer the initial questions before class, to the best of your ability. Once class has started you may rework your answers as a team for the initial part of the assignment. 

<a id="top"></a>
## Contents
* <a href="#Loading">Loading the Data</a>
* <a href="#svm">Linear SVMs</a>
* <a href="#svm_using">Using Linear SVMs</a>
* <a href="#nonlinear">Non-linear SVMs</a>

________________________________________________________________________________________________________

<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Loading the Data
Please run the following code to read in the "olivetti faces" dataset from sklearn's data loading module. 

This will load the data into the variable `ds`. `ds` is a `bunch` object with fields like `ds.data` and `ds.target`. The field `ds.data` is a numpy matrix of the continuous features in the dataset. **The object is not a pandas dataframe. It is a numpy matrix.** Each row is a set of observed instances, each column is a different feature. It also has a field called `ds.target` that is an integer value we are trying to predict (i.e., a specific integer represents a specific person). Each entry in `ds.target` is a label for each row of the `ds.data` matrix. 

In [5]:
# fetch the images for the dataset
# this will take a long time the first run because it needs to download
# after the first time, the dataset will be save to your disk (in sklearn package somewhere) 
# if this does not run, you may need additional libraries installed on your system (install at your own risk!!)
from sklearn.datasets import fetch_lfw_people

lfw_people = fetch_lfw_people(min_faces_per_person=20, resize=None)

In [6]:
# get some of the specifics of the dataset
X = lfw_people.data
y = lfw_people.target
names = lfw_people.target_names

n_samples, n_features = X.shape
_, h, w = lfw_people.images.shape
n_classes = len(names)

print("n_samples: {}".format(n_samples))
print("n_features: {}".format(n_features))
print("n_classes: {}".format(n_classes))
print("Original Image Sizes {} by {}".format(h,w))
print (125*94) # the size of the images are the size of the feature vectors

n_samples: 3023
n_features: 11750
n_classes: 62
Original Image Sizes 125 by 94
11750


**Question 1:** For the faces dataset, describe what the data represents? That is, what is each column? What is each row? What do the unique class values represent?



"Faces in the wild". The target vector (or class) corresponds to the name of a person, each with a unique value. Each row is a picture of a persons face. Each entry in the matrix corresponds to a pixel value scaled to fit between 0 and 1 for each rgb channel.



___

<a id="svm"></a>
<a href="#top">Back to Top</a>
## Linear Support Vector Machines

**Question 2:** If we were to train a linear Support Vector Machine (SVM) upon the faces data, how many parameters would need to be optimized in the model? That is, how many coefficients would need to be calculated?


You need to optimize a weight for each feature plus one bias term for each of the 62 classes.


In [7]:
# Enter any scratchwork or calculations here
total_optimization_parameters = 62*(11750 + 1)
total_optimization_parameters

728562

**Question 3:** 
- **Part A:** Given the number of parameters calculated above, would you expect the model to train quickly using **batch optimization techniques**? Why or why not? 
- **Part B:** Is there a way to reduce training time?
- **Part C:** If we transformed the X data using principle components analysis (PCA) with 100 components, how many parameters would we need to find for a linear Support Vector Machine (SVM)?

*Enter you answer here (double click)*


A. No, too much data. Calculating the gradient for every one of the sample each iteration is computationally expensive.

B. Yes PCA, quasi newton, stochastic gradient descent, or even grouping by classes could reduce training time.

C. 6262 = 62*(100 + 1)
___

In [8]:
# Enter any scratchwork or calculations here

print('Part C. With 100 features: ', '??')
62*(100 + 1)

Part C. With 100 features:  ??


6262

___
<a id="svm_using"></a>
<a href="#top">Back to Top</a>

# Using Linear SVMs

**Exercise 1:** Use the block of code below to check if the number of parameters you calculated is equal to the number of parameters returned by `sklearn`'s implementation of the Linear SVM. **Was your calculation correct? If different, can you think of a reason why the parameters would not match?**

In [9]:
from sklearn.svm import LinearSVC
from sklearn.decomposition import RandomizedPCA

n_components = 100
pca = RandomizedPCA(n_components=n_components)
Xpca = pca.fit_transform(X)

clf = LinearSVC()
clf.fit(Xpca,y)

/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:57: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.
  warnings.warn(msg, category=DeprecationWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [15]:
#===================================================================
# Enter your code below to calculate the number of parameters in the model 

clf.coef_.shape
clf.intercept_.shape
clf.coef_.shape[0]*(clf.coef_.shape[1]) + clf.intercept_.shape[0]
#===================================================================

6262

___
**Exercise 2:** Use the starter code below to calculate two quantities: 
- **Part A.:** The overall accuracy of the trained linear svm on the training set
- **Part B.:** The *mean, standard deviation, maximum, and minimum* of the **accuracy per class** on the training set

You might be interested in the following documentation of the confusion matrix calculated by `scikit-learn`:
- http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

And an example matrix returned by the confusion matrix function:
<img src="http://scikit-learn.org/stable/_images/sphx_glr_plot_confusion_matrix_001.png",width=400,height=400>

In [47]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np

yhat = clf.predict(Xpca)

#===================================================
# Enter your code below

cnf_matrix = confusion_matrix(y, yhat)


normalized_cm = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]

d = np.diagonal(normalized_cm)

mean = np.mean(d)
std = np.std(d)
maximum = np.max(d)
minimum = np.min(d)



print('Overall Accuracy is %s' % accuracy_score(y, yhat))

print('The class accuracy is ',str(mean), '+-', str(std), end=' ')
print('(min,max) (',str(minimum),',' , str(maximum),')')

#===================================================

Overall Accuracy is 0.8819053919947073
The class accuracy is  0.8949871083221426 +- 0.12065998790862242 (min,max) ( 0.46153846153846156 , 1.0 )


___
<a id="nonlinear"></a>
<a href="#top">Back to Top</a>

# Non-linear SVMs
Now let's explore the use of non-linear svms. More explicitly, using different kernels. Take a look at the example training and testing code below for the non-linear SVM. All parameters are left as default, except we change the kernel to be `rbf`. Run the block of code below.



In [50]:
from sklearn.svm import SVC
import time

clf = SVC(kernel='rbf')
%time clf.fit(Xpca,y)
yhat = clf.predict(Xpca)
print('Overall Accuracy of rbf kernal is ',accuracy_score(y,yhat))

CPU times: user 3.36 s, sys: 13 ms, total: 3.38 s
Wall time: 3.39 s
Overall Accuracy of rbf kernal is  0.9361561362884552
CPU times: user 2.22 s, sys: 3.89 ms, total: 2.22 s
Wall time: 2.22 s
Overall Accuracy of poly kernal is  0.9748594111809461
CPU times: user 1.04 s, sys: 805 µs, total: 1.04 s
Wall time: 1.04 s
Overall Accuracy of sigmoid kernal is  0.21634138273238504


___
**Exercise 3:** Use the starter code from above to calculate the accuracy for three different non-linear SVM kernels. That is, repeat the code above for different `kernel` parameters. **Only use non-linear kernels.  Which kernel is most accurate with the default parameters?**

You might be interested in the documentation of the scikit-learn SVM implementation, available here:
- http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [63]:
#===================================================
# Enter your code below

clf_rbf = SVC(kernel='rbf')
%time clf_rbf.fit(Xpca,y)
yhat = clf_rbf.predict(Xpca)
print('Overall Accuracy of rbf kernal is ',accuracy_score(y,yhat))

clf_poly = SVC(kernel='poly')
%time clf_poly.fit(Xpca,y)
yhat = clf_poly.predict(Xpca)
print('Overall Accuracy of poly kernal is ',accuracy_score(y,yhat))

clf_sigmoid = SVC(kernel='sigmoid')
%time clf_sigmoid.fit(Xpca,y)
yhat = clf_sigmoid.predict(Xpca)
print('Overall Accuracy of sigmoid kernal is ',accuracy_score(y,yhat))

#===================================================

CPU times: user 3.29 s, sys: 8.97 ms, total: 3.3 s
Wall time: 3.3 s
Overall Accuracy of rbf kernal is  0.9361561362884552
CPU times: user 2.18 s, sys: 2.25 ms, total: 2.18 s
Wall time: 2.18 s
Overall Accuracy of poly kernal is  0.9748594111809461
CPU times: user 1.05 s, sys: 1.44 ms, total: 1.05 s
Wall time: 1.05 s
Overall Accuracy of sigmoid kernal is  0.21634138273238504


The most accurate kernel with default paramters is poly kernal.


___
**Exercise 4:** Choose the **most accurate kernel** and manipulate the settings for `gamma` to make the classification more accurate. 
- **Part A:** How accurate can you make it? 
- **Part B:** Would you expect the results to generalize well? Why or why not?

In [70]:
#===================================================
# Enter your code below
from sklearn.model_selection import GridSearchCV

#Create a dictionary of possible parameters
params_grids = {'gamma': [0.0001, 0.001, 0.01, 0.1], 'kernel': ['poly'] }

#Create the GridSearchCV object
grid_clf = GridSearchCV(SVC(class_weight='balanced'), params_grids)

#Fit the data with the best possible parameters
grid_clf = grid_clf.fit(Xpca, y) #clf_poly.fit(Xpca, y) #

#Print the best estimator with it's parameters
kern = 'poly'
g = grid_clf.best_estimator_.gamma

yhat = SVC(kernel=kern, gamma=g).fit(Xpca,y).predict(Xpca)
print('Overall Accuracy is ', accuracy_score(y,yhat)) 

#===================================================

Overall Accuracy is  1.0



A. The highest accuracy we could achieve was 1.0

B. We would not expect this to generalize because we are basically overfitting our dataset.



___
**Final Question:** Using the most accurate non-linear SVM you found in the previous question, how many parameter coefficients does the trained model contain?

As a result of our using a kernel trick in order to compute our non-linear SVM there is not necessarily a concrete answer for what constitutes a parameter. In this case, we are considering a parameter to be a 'dual coefficient,' because they give you the weights of the support vectors in each of the respective binary classification problems.

61


In [77]:
#===================================================
# Enter any scratchwork calculations you need below
yhat1 = SVC(kernel=kern, gamma=g).fit(Xpca,y)
print(len(yhat1.dual_coef_))

61


________________________________________________________________________________________________________

That's all! Please **save (make sure you saved!!!) and upload your rendered notebook** and please include **team member names** in the notebook submission.